In [1]:
# Install required packages if not already installed
import subprocess
import sys

def install_package(package):
    try:
        __import__(package)
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install required packages
packages = ['requests', 'beautifulsoup4', 'tqdm']
for package in packages:
    install_package(package)

print("All required packages are installed!")


Installing requests...
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached charset_normalizer-3.4.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
Using cached certifi-2025.8.3-py3-none-any.whl (161 kB)
Using cached charset_normalizer-3.4.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl (151 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
Installing beautifulsoup4...
  Using cached soupsieve-2.8-py3-none-any.whl.metadata (4.6 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 1.3 MB/s eta 0:00

In [2]:
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin, unquote
import time
from tqdm import tqdm

# URL to scrape
base_url = "https://archive.org/download/the-history-of-rome"
print(f"Scraping MP3 files from: {base_url}")

# Create downloads directory
downloads_dir = "history_of_rome_episodes"
os.makedirs(downloads_dir, exist_ok=True)
print(f"Downloads will be saved to: {downloads_dir}")

# Get the page content
response = requests.get(base_url)
response.raise_for_status()

# Parse the HTML
soup = BeautifulSoup(response.content, 'html.parser')

# Find all links that end with .mp3
mp3_links = []
for link in soup.find_all('a', href=True):
    href = link['href']
    if href.endswith('.mp3'):
        # Create full URL
        full_url = urljoin(base_url + "/", href)
        # Decode URL-encoded characters in filename
        filename = unquote(href)
        mp3_links.append((full_url, filename))

print(f"Found {len(mp3_links)} MP3 files:")
for i, (url, filename) in enumerate(mp3_links[:5], 1):  # Show first 5
    print(f"{i}. {filename}")
if len(mp3_links) > 5:
    print(f"... and {len(mp3_links) - 5} more")

mp3_links

Scraping MP3 files from: https://archive.org/download/the-history-of-rome
Downloads will be saved to: history_of_rome_episodes
Found 192 MP3 files:
1. 20070728 - 001- In the Beginning.mp3
2. 20100225 - 002- Youthful Indiscretions.mp3
3. 20100225 - 003a- The Seven Kings of Rome.mp3
4. 20100225 - 003b- The Seven Kings of Rome.mp3
5. 20100225 - 004- The Public Thing.mp3
... and 187 more


[('https://archive.org/download/the-history-of-rome/20070728%20-%20001-%20In%20the%20Beginning.mp3',
  '20070728 - 001- In the Beginning.mp3'),
 ('https://archive.org/download/the-history-of-rome/20100225%20-%20002-%20Youthful%20Indiscretions.mp3',
  '20100225 - 002- Youthful Indiscretions.mp3'),
 ('https://archive.org/download/the-history-of-rome/20100225%20-%20003a-%20The%20Seven%20Kings%20of%20Rome.mp3',
  '20100225 - 003a- The Seven Kings of Rome.mp3'),
 ('https://archive.org/download/the-history-of-rome/20100225%20-%20003b-%20The%20Seven%20Kings%20of%20Rome.mp3',
  '20100225 - 003b- The Seven Kings of Rome.mp3'),
 ('https://archive.org/download/the-history-of-rome/20100225%20-%20004-%20The%20Public%20Thing.mp3',
  '20100225 - 004- The Public Thing.mp3'),
 ('https://archive.org/download/the-history-of-rome/20100225%20-%20005-%20Trials%20and%20Tribunlations.mp3',
  '20100225 - 005- Trials and Tribunlations.mp3'),
 ('https://archive.org/download/the-history-of-rome/20100225%20-%20006

In [3]:
def download_file(url, filename, downloads_dir):
    """Download a file with progress bar and error handling"""
    filepath = os.path.join(downloads_dir, filename)
    
    # Skip if file already exists
    if os.path.exists(filepath):
        print(f"Skipping {filename} - already exists")
        return True
    
    try:
        # Stream download with progress bar
        response = requests.get(url, stream=True)
        response.raise_for_status()
        
        # Get file size for progress bar
        total_size = int(response.headers.get('content-length', 0))
        
        with open(filepath, 'wb') as file, tqdm(
            desc=filename[:50],  # Truncate long filenames for display
            total=total_size,
            unit='B',
            unit_scale=True,
            unit_divisor=1024,
        ) as pbar:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    file.write(chunk)
                    pbar.update(len(chunk))
        
        print(f"✓ Downloaded: {filename}")
        return True
        
    except Exception as e:
        print(f"✗ Failed to download {filename}: {str(e)}")
        # Remove partial file if it exists
        if os.path.exists(filepath):
            os.remove(filepath)
        return False

# Download all MP3 files
print(f"\nStarting download of {len(mp3_links)} MP3 files...")
print("=" * 60)

successful_downloads = 0
failed_downloads = 0

for i, (url, filename) in enumerate(mp3_links, 1):
    print(f"\n[{i}/{len(mp3_links)}] Downloading: {filename}")
    
    if download_file(url, filename, downloads_dir):
        successful_downloads += 1
    else:
        failed_downloads += 1
    
    # Small delay to be respectful to the server
    time.sleep(0.5)

print("\n" + "=" * 60)
print(f"Download Summary:")
print(f"✓ Successful: {successful_downloads}")
print(f"✗ Failed: {failed_downloads}")
print(f"📁 Files saved to: {os.path.abspath(downloads_dir)}")

# List downloaded files
downloaded_files = [f for f in os.listdir(downloads_dir) if f.endswith('.mp3')]
print(f"\nDownloaded {len(downloaded_files)} MP3 files:")
for file in sorted(downloaded_files)[:10]:  # Show first 10
    file_path = os.path.join(downloads_dir, file)
    file_size = os.path.getsize(file_path) / (1024 * 1024)  # Size in MB
    print(f"  • {file} ({file_size:.1f} MB)")
if len(downloaded_files) > 10:
    print(f"  ... and {len(downloaded_files) - 10} more files")



Starting download of 192 MP3 files...

[1/192] Downloading: 20070728 - 001- In the Beginning.mp3


20070728 - 001- In the Beginning.mp3: 100%|██████████| 5.40M/5.40M [00:01<00:00, 5.66MB/s]


✓ Downloaded: 20070728 - 001- In the Beginning.mp3

[2/192] Downloading: 20100225 - 002- Youthful Indiscretions.mp3


20100225 - 002- Youthful Indiscretions.mp3: 100%|██████████| 5.47M/5.47M [00:02<00:00, 2.38MB/s]


✓ Downloaded: 20100225 - 002- Youthful Indiscretions.mp3

[3/192] Downloading: 20100225 - 003a- The Seven Kings of Rome.mp3


...

✓ Downloaded: 20130530 - Update- One Year Later.mp3

[191/192] Downloading: 20130916 - Revolutions Launch.mp3


20130916 - Revolutions Launch.mp3: 100%|██████████| 247k/247k [00:00<00:00, 828kB/s] 


✓ Downloaded: 20130916 - Revolutions Launch.mp3

[192/192] Downloading: 20170727 - The Storm Before The Storm_ Chapter 1- The Beasts of Italy.mp3


20170727 - The Storm Before The Storm_ Chapter 1- : 100%|██████████| 76.5M/76.5M [00:20<00:00, 3.84MB/s]


✓ Downloaded: 20170727 - The Storm Before The Storm_ Chapter 1- The Beasts of Italy.mp3

Download Summary:
✓ Successful: 192
✗ Failed: 0
📁 Files saved to: /home/gary/history_of_rome_epidode_search/history_of_rome_episodes

Downloaded 192 MP3 files:
  • 20070728 - 001- In the Beginning.mp3 (5.4 MB)
  • 20100225 - 002- Youthful Indiscretions.mp3 (5.5 MB)
  • 20100225 - 003a- The Seven Kings of Rome.mp3 (7.3 MB)
  • 20100225 - 003b- The Seven Kings of Rome.mp3 (8.3 MB)
  • 20100225 - 004- The Public Thing.mp3 (6.8 MB)
  • 20100225 - 005- Trials and Tribunlations.mp3 (7.3 MB)
  • 20100225 - 006- The Twelve Tables.mp3 (7.3 MB)
  • 20100225 - 007- The Roman Washington.mp3 (5.6 MB)
  • 20100225 - 008- Decades of Gloom.mp3 (7.1 MB)
  • 20100225 - 009- A Trojan War.mp3 (6.4 MB)
  ... and 182 more files
